# ЗАДАНИЕ №1


In [29]:
import requests
import json
import urllib3
import ssl
from requests.adapters import HTTPAdapter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import ai
from google.colab import userdata

# Убираем не нужные предупреждения
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [30]:
# ----------- GigaChat KEY -----------
CLIENT_ID_SECRET_BASE64 = userdata.get('GIGACHAT_KEY')

AUTH_URL = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
CHAT_URL = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

# ----------- OpenRouter KEY -----------
OPENROUTER_KEY =userdata.get('OPENROUTER_KEY')
OPENROUTER_MODEL = "mistralai/mistral-small-3.1-24b-instruct:free"

In [31]:
# Позволяет работать с самоподписанными сертификатами без ошибок
class SSLIgnoreAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
        kwargs["ssl_context"] = ctx
        return super().init_poolmanager(*args, **kwargs)

# Создаём сессию requests с нашим SSL адаптером
session = requests.Session()
session.mount("https://", SSLIgnoreAdapter())
session.mount("http://", SSLIgnoreAdapter())

In [32]:
# Функция генерации промпта для всех моделей
def make_prompt(text):
    return f"""
Проанализируй отзыв о дроне. Верни строго JSON одной строкой:
{{"sentiment": "положительный/нейтральный/отрицательный",
 "main_topic": "...",
 "issue": "...",
 "rating": число от 1 до 5}}
Отзыв: <<{text}>>
"""

In [33]:
# Получение токена
def get_gigachat_token():
    headers = {
        "Authorization": f"Basic {CLIENT_ID_SECRET_BASE64}",
        "RqUID": "123e4567-e89b-12d3-a456-426614174000",
        "Content-Type": "application/x-www-form-urlencoded",
    }
    data = {"scope": "GIGACHAT_API_PERS"}
    r = session.post(AUTH_URL, headers=headers, data=data, verify=False)
    return r.json()["access_token"]

In [34]:
def analyze_gigachat(text):
    prompt = make_prompt(text)
    token = get_gigachat_token()

    headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}
    payload = {
        "model": "GigaChat",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 300,
    }

    r = session.post(CHAT_URL, headers=headers, json=payload, verify=False)

    try:
        raw = r.json()["choices"][0]["message"]["content"]
        parsed = json.loads(raw[raw.find("{"):raw.rfind("}")+1])
    except:
        parsed = {"sentiment": None, "main_topic": None, "issue": None, "rating": None}

    return parsed

In [35]:
def analyze_colab(text):
    prompt = make_prompt(text)
    response = ai.generate_text(prompt)

    try:
        parsed = json.loads(response[response.find("{"):response.rfind("}")+1])
    except:
        parsed = {"sentiment": None, "main_topic": None, "issue": None, "rating": None}

    return parsed

In [36]:
def analyze_openrouter(text):
    prompt = make_prompt(text)

    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {OPENROUTER_KEY}",
            "HTTP-Referer": "colab-app",
            "X-Title": "colab-app",
        },
        data=json.dumps({
            "model": OPENROUTER_MODEL,
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 400
        })
    )

    res = response.json()

    if "choices" not in res:
        return {"sentiment": None, "main_topic": None, "issue": None, "rating": None}

    raw = res["choices"][0]["message"]["content"]

    try:
        parsed = json.loads(raw[raw.find("{"):raw.rfind("}")+1])
    except:
        parsed = {"sentiment": None, "main_topic": None, "issue": None, "rating": None}

    return parsed

In [37]:
reviews = [
    "легко управляется. камера хорошо передаёт изображение. устойчив к падениям ",
    "Долго решался какой покупать для начала fpv дрон. Данный дрон как для первого опыта, я считаю хорошим вложением, полеты соответствуют указанным( на минимальных скоростях время полета у меня составило на одной батарее 12 минут) . Не боится ударов. Легко соединяется все элементы (шлем, п/у, очки) . Недостатков еще не обнаружил. ",
    "Классный аппарат! Дроном очень доволен, его фото и видео возможностями. Аккумуляторы позволяют находится в воздухе весьма продолжительное время. Для создателей контента рекомендую! Как игрушка штука интересная, но есть модели, которые будут не хуже и стоить дешевле ",
    "Отличный дрон для введения в тему. Ну и для профи тоже интерес, ведь не каждый дрон в карман помещается .... Доволен покупкой ",
    "В первый раз попался брак, неисправность шлейфа либо платы. Отсутствовало изображение с камеры.  Перезаказал здесь же, во второй раз все отлично. Доставка быстрая, возврат одобрили тоже быстро. Стоит отметить отличный сервис продавца. Ответили в течении пяти минут, несмотря на позднее время. Предложили несколько вариантов решения проблемы.  Продавца рекомендую. В дальнейшем буду покупать только здесь. ",
    "Динамик и сброс в комплекте, думал не пригодится, но это одно из лучших приспособлений. Дрон устойчивый , дальше чем на км не отпускаю. Большая вероятность потери сигнала без усилителя.",
    "Батарейка держит 3 минуты от силы Могли бы не одну батарею ложить а три чтобы летал 10 минут как заявлено на упаковки",
    "Камера не передаёт видео, так как не получается подключиться к приложению  (нет связи), не подключается к вайфаю, не показывает сеть. Брала в подарок, подарила и получается что подарила сломанную вещь",
    "Купил пульт DJI RC Pro, качество сборки и материалов на высоте, сразу видно, что это оригинальный DJI. Большой плюс - передача картинки на большое расстояние. Экран яркий и четкий, все команды и настройки видны хорошо.",
    "Приехал квадрокоптер, именно DJI Mini 4 Pro Fly More Combo Plus с усиленными батареями, в упаковке, но активированный в Китае в августе месяце, судя по отзывам они сейчас практически все приходят активированными... Пульт сбасываешь к заводским настройкам, обновляешь и настраиваешь под себя. Батареи проверили наработка 0 часов, серийники бьются на коробке и на самом дроне. В полете прверили, все функции отлично работают, покупкой довольны... С промиком и бонусами получилось очень выгодно)"
]

In [38]:
rows = []

for r in reviews:
    print("\nОТЗЫВ:", r, "\n")

    colab_res = analyze_colab(r)
    giga_res = analyze_gigachat(r)
    openrouter_res = analyze_openrouter(r)

    print("COLAB:", colab_res)
    print("GIGACHAT:", giga_res)
    print("OPENROUTER:", openrouter_res)

    rows.append({
        "review": r,
        "colab_sent": colab_res["sentiment"],
        "giga_sent": giga_res["sentiment"],
        "open_sent": openrouter_res["sentiment"],
    })

    print("\n--------------------------------------")



ОТЗЫВ: легко управляется. камера хорошо передаёт изображение. устойчив к падениям  

COLAB: {'sentiment': 'положительный', 'main_topic': 'Качество и удобство использования дрона', 'issue': None, 'rating': 5}
GIGACHAT: {'sentiment': 'положительный', 'main_topic': 'качество управления и камеры', 'issue': '', 'rating': 5}
OPENROUTER: {'sentiment': None, 'main_topic': None, 'issue': None, 'rating': None}

--------------------------------------

ОТЗЫВ: Долго решался какой покупать для начала fpv дрон. Данный дрон как для первого опыта, я считаю хорошим вложением, полеты соответствуют указанным( на минимальных скоростях время полета у меня составило на одной батарее 12 минут) . Не боится ударов. Легко соединяется все элементы (шлем, п/у, очки) . Недостатков еще не обнаружил.  

COLAB: {'sentiment': 'положительный', 'main_topic': 'FPV дрон для новичков, время полета, прочность, легкость подключения', 'issue': 'отсутствуют', 'rating': 5}
GIGACHAT: {'sentiment': 'положительный', 'main_topic': 